In [4]:
import pandas as pd

# Load the dataset
file_path = '/Users/deveshattri/Desktop/MegaMinds Internship/IMDB Dataset.csv'
imdb_dataset = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(imdb_dataset.head())


                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [5]:
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

# Remove duplicates and null values
imdb_dataset = imdb_dataset.drop_duplicates().dropna()

# Split the dataset into training and testing sets
train_data, test_data = train_test_split(imdb_dataset, test_size=0.2, random_state=42)

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the dataset
def tokenize_data(data):
    return tokenizer(data['review'].tolist(), padding='max_length', truncation=True, return_tensors="pt")

train_tokens = tokenize_data(train_data)
test_tokens = tokenize_data(test_data)


In [6]:
import torch
from torch.utils.data import Dataset, DataLoader

class IMDBDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_labels = train_data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0).tolist()
test_labels = test_data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0).tolist()

train_dataset = IMDBDataset(train_tokens, train_labels)
test_dataset = IMDBDataset(test_tokens, test_labels)


In [7]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
import torch

# Check for available device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Load the BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.to(device)

# Define training arguments with a smaller batch size
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,  # Reduce batch size
    per_device_eval_batch_size=4,   # Reduce batch size
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    fp16=False,  # Ensure mixed precision training is disabled
    no_cuda=True,  # Disable CUDA as MPS is used
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
# Train the model
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/deveshattri/Desktop/MegaMinds Internship/.venv/lib/python3.10/site-packages/transformers/training_args.py:1489: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


  0%|          | 0/600 [00:00<?, ?it/s]

/var/folders/p1/5swwv58x6zx6zvpgsynmmnjm0000gn/T/ipykernel_77068/2609057923.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'loss': 0.7012, 'grad_norm': 6.774988174438477, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.05}
{'loss': 0.6821, 'grad_norm': 11.505744934082031, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.1}
{'loss': 0.6903, 'grad_norm': 7.213844299316406, 'learning_rate': 3e-06, 'epoch': 0.15}
{'loss': 0.708, 'grad_norm': 12.112350463867188, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.2}
{'loss': 0.7023, 'grad_norm': 8.102845191955566, 'learning_rate': 5e-06, 'epoch': 0.25}
{'loss': 0.6891, 'grad_norm': 3.6958343982696533, 'learning_rate': 6e-06, 'epoch': 0.3}
{'loss': 0.6693, 'grad_norm': 3.397529125213623, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.35}
{'loss': 0.6855, 'grad_norm': 7.077503681182861, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.4}
{'loss': 0.7244, 'grad_norm': 11.456257820129395, 'learning_rate': 9e-06, 'epoch': 0.45}
{'loss': 0.6378, 'grad_norm': 9.263278007507324, 'learning_rate': 1e-05, 'epoch': 0.5}
{'loss': 0.6435, 'grad_norm': 5.17078971

/var/folders/p1/5swwv58x6zx6zvpgsynmmnjm0000gn/T/ipykernel_77068/2609057923.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'loss': 0.3334, 'grad_norm': 0.2813985347747803, 'learning_rate': 4.5e-05, 'epoch': 2.55}
{'loss': 0.2375, 'grad_norm': 0.09375448524951935, 'learning_rate': 4e-05, 'epoch': 2.6}
{'loss': 0.2341, 'grad_norm': 17.614187240600586, 'learning_rate': 3.5e-05, 'epoch': 2.65}
{'loss': 0.1503, 'grad_norm': 26.835676193237305, 'learning_rate': 3e-05, 'epoch': 2.7}
{'loss': 0.2835, 'grad_norm': 0.9994214177131653, 'learning_rate': 2.5e-05, 'epoch': 2.75}
{'loss': 0.0876, 'grad_norm': 0.5390771627426147, 'learning_rate': 2e-05, 'epoch': 2.8}
{'loss': 0.1757, 'grad_norm': 0.03979887068271637, 'learning_rate': 1.5e-05, 'epoch': 2.85}
{'loss': 0.1382, 'grad_norm': 0.041395846754312515, 'learning_rate': 1e-05, 'epoch': 2.9}
{'loss': 0.002, 'grad_norm': 0.17792600393295288, 'learning_rate': 5e-06, 'epoch': 2.95}
{'loss': 0.1717, 'grad_norm': 55.51280975341797, 'learning_rate': 0.0, 'epoch': 3.0}
{'train_runtime': 3877.0092, 'train_samples_per_second': 0.618, 'train_steps_per_second': 0.155, 'train_lo

TrainOutput(global_step=600, training_loss=0.39501675105964146, metrics={'train_runtime': 3877.0092, 'train_samples_per_second': 0.618, 'train_steps_per_second': 0.155, 'total_flos': 630677199697920.0, 'train_loss': 0.39501675105964146, 'epoch': 3.0})

In [20]:
# Save the model
model.save_pretrained('./saved_model')

In [8]:
from textattack.datasets import Dataset

# Create a custom dataset for TextAttack
class CustomDataset(Dataset):
    def __init__(self, reviews, labels):
        self.reviews = reviews
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.reviews[idx], self.labels[idx]

# Prepare the dataset for TextAttack
train_reviews = train_data['review'].tolist()
train_labels = train_data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0).tolist()

train_dataset_attack = CustomDataset(train_reviews, train_labels)


In [9]:
from textattack.attack_recipes import TextFoolerJin2019
from textattack.models.wrappers import HuggingFaceModelWrapper

# Wrap the model
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Generate adversarial examples using TextFooler
attack = TextFoolerJin2019.build(model_wrapper)


textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [17]:
from textattack import Attacker, Attack, AttackArgs, datasets, models, transformations, constraints, goal_functions, search_methods
from textattack.models.wrappers import HuggingFaceModelWrapper
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load pre-trained model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb")
tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")

# Wrap the model using TextAttack's HuggingFaceModelWrapper
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Define your attack here (customize as per your requirement)
attack = Attack(
    goal_function=goal_functions.UntargetedClassification(model_wrapper),
    constraints=[constraints.pre_transformation.StopwordModification()],
    transformation=transformations.WordSwapEmbedding(),
    search_method=search_methods.GreedyWordSwapWIR()
)

# Load dataset using HuggingFace datasets library and wrap it with TextAttack's HuggingFaceDataset
train_dataset_attack = datasets.HuggingFaceDataset("imdb", split="train")

# Initialize the attacker with attack arguments
attack_args = AttackArgs(num_examples=5)  # Customize the number of examples
attacker = Attacker(attack, train_dataset_attack, attack_args)

# Perform the attack on the dataset
adversarial_examples = attacker.attack_dataset()


textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Loading datasets dataset imdb, split train.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  15
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): StopwordModification
  (is_black_box):  True
) 



--------------------------------------------- Result 1 ---------------------------------------------


  0%|          | 0/5 [03:02<?, ?it/s]


[[Negative (100%)]] --> [[Positive (52%)]]

I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex an

--------------------------------------------- Result 2 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (54%)]]

"I Am Curious: Yellow" is a risible and [[pretentious]] steaming [[pile]]. It doesn't [[matter]] what one's political views are because this [[film]] can [[hardly]] be taken [[seriously]] on any level. As for the claim that frontal male nudity is an automatic NC-17, that isn't true. I've [[seen]] R-rated films with male [[nudity]]. Granted, they only offer some fleeting views, but where are the R-rated films with gaping vulvas and flapping labia? Nowhere, because they don't exist. The same goes for those crappy [[cable]] shows: schlongs swinging in the breeze but not a clitoris in sight. [[And]] those pretentious [[indie]] movies [[like]] The Brown [[Bunny]], in which we're treated to the site of Vincent Gallo's throbbing [[johnson]], but not a trace of pink visible on Chloe Sevigny. [[Before]] [[crying]] (or [[implying]]) "double-standard" in

--------------------------------------------- Result 3 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (69%)]]

If only to [[avoid]] making this type of film in the [[future]]. This film is interesting as an experiment but tells no cogent story.<br /><br />One [[might]] feel virtuous for sitting thru it because it touches on so many IMPORTANT issues but it does so without any discernable motive. The viewer comes away with no new perspectives (unless one comes up with one while one's mind wanders, as it will invariably do during this [[pointless]] film).<br /><br />One might better spend one's time staring out a window at a tree growing.<br /><br />

If only to [[evita]] making this type of film in the [[futures]]. This film is interesting as an experiment but tells no cogent story.<br /><br />One [[conceivably]] feel virtuous for sitting thru it because it touches on so many IMPORTANT issues but it does so without any discernable motive. The viewer come

--------------------------------------------- Result 4 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (99%)]]

This film was probably inspired by Godard's Masculin, féminin and I urge you to see that film instead.<br /><br />The film has two strong elements and those are, (1) the realistic acting (2) the impressive, undeservedly good, [[photo]]. Apart from that, what strikes me most is the endless stream of [[silliness]]. Lena Nyman has to be most annoying actress in the [[world]]. She acts so stupid and with all the nudity in this film,...it's unattractive. Comparing to Godard's film, intellectuality has been replaced with stupidity. Without going too far on this subject, I would say that follows from the difference in ideals between the French and the Swedish society.<br /><br />A movie of its time, and place. 2/10.

This film was probably inspired by Godard's Masculin, féminin and I urge you to see that film instead.<br /><br />The film has two stro

[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5: 100%|██████████| 5/5 [08:12<00:00, 98.54s/it]

--------------------------------------------- Result 5 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (50%)]]

Oh, brother...after hearing about this [[ridiculous]] [[film]] for umpteen years all I can [[think]] of is that old [[Peggy]] [[Lee]] [[song]]..<br /><br />"Is that all there is??" ...[[I]] was just an early teen when this [[smoked]] [[fish]] [[hit]] the U.S. [[I]] was too [[young]] to get in the [[theater]] (although [[I]] did manage to sneak into "Goodbye [[Columbus]]"). Then a [[screening]] at a [[local]] [[film]] [[museum]] [[beckoned]] - Finally I could see this film, [[except]] now [[I]] was as [[old]] as my parents were when they schlepped to see it!!<br /><br />The [[ONLY]] [[reason]] this film was not condemned to the [[anonymous]] sands of [[time]] was because of the obscenity case sparked by its U.S. release. MILLIONS of people [[flocked]] to this stinker, thinking they were going to see a [[sex]] film...[[Instead]], they got lots o

In [24]:
# Step 8.1: Retrieve Adversarial Examples and Convert to Suitable Format
adversarial_reviews = [ex.perturbed_text() for ex in adversarial_examples]
adversarial_labels = [ex.original_result.output for ex in adversarial_examples]

# Combine original and adversarial examples
combined_reviews = train_reviews + adversarial_reviews
combined_labels = train_labels + adversarial_labels

# Step 8.2: Create Combined Dataset
combined_tokens = tokenizer(combined_reviews, padding='max_length', truncation=True, return_tensors="pt")
combined_labels = torch.tensor(combined_labels)

combined_dataset = IMDBDataset(combined_tokens, combined_labels)

# Step 8.3: Retrain the Model
# Define new training arguments
combined_training_args = TrainingArguments(
    output_dir='./combined_results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./combined_logs',
    logging_steps=10,
)

# Initialize the Trainer with the combined dataset
combined_trainer = Trainer(
    model=model,
    args=combined_training_args,
    train_dataset=combined_dataset,
    eval_dataset=test_dataset,
)

# Retrain the model
combined_trainer.train()

# Save the retrained model
model.save_pretrained('./retrained_model')

  0%|          | 0/603 [00:00<?, ?it/s]

/var/folders/p1/5swwv58x6zx6zvpgsynmmnjm0000gn/T/ipykernel_77068/2609057923.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/var/folders/p1/5swwv58x6zx6zvpgsynmmnjm0000gn/T/ipykernel_77068/2609057923.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


{'loss': 0.2314, 'grad_norm': 14.510295867919922, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.05}
{'loss': 0.1547, 'grad_norm': 4.589311599731445, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.1}
{'loss': 0.4386, 'grad_norm': 179.24148559570312, 'learning_rate': 3e-06, 'epoch': 0.15}
{'loss': 0.2067, 'grad_norm': 224.6477813720703, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.2}
{'loss': 0.303, 'grad_norm': 0.24597696959972382, 'learning_rate': 5e-06, 'epoch': 0.25}
{'loss': 0.1911, 'grad_norm': 95.0750732421875, 'learning_rate': 6e-06, 'epoch': 0.3}
{'loss': 0.0034, 'grad_norm': 0.15990053117275238, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.35}
{'loss': 0.0085, 'grad_norm': 0.5522549748420715, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.4}
{'loss': 0.2448, 'grad_norm': 112.57675170898438, 'learning_rate': 9e-06, 'epoch': 0.45}
{'loss': 0.0241, 'grad_norm': 0.00696523254737258, 'learning_rate': 1e-05, 'epoch': 0.5}
{'loss': 0.4238, 'grad_norm': 0.024

/var/folders/p1/5swwv58x6zx6zvpgsynmmnjm0000gn/T/ipykernel_77068/2609057923.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/var/folders/p1/5swwv58x6zx6zvpgsynmmnjm0000gn/T/ipykernel_77068/2609057923.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


{'loss': 0.3268, 'grad_norm': 0.09010977298021317, 'learning_rate': 4.514563106796117e-05, 'epoch': 2.54}
{'loss': 0.2417, 'grad_norm': 0.46364179253578186, 'learning_rate': 4.029126213592233e-05, 'epoch': 2.59}
{'loss': 0.0587, 'grad_norm': 1.9972138404846191, 'learning_rate': 3.54368932038835e-05, 'epoch': 2.64}
{'loss': 0.2708, 'grad_norm': 246.81240844726562, 'learning_rate': 3.058252427184466e-05, 'epoch': 2.69}
{'loss': 0.131, 'grad_norm': 0.004374051000922918, 'learning_rate': 2.5728155339805826e-05, 'epoch': 2.74}
{'loss': 0.0218, 'grad_norm': 0.7514193654060364, 'learning_rate': 2.0873786407766992e-05, 'epoch': 2.79}
{'loss': 0.009, 'grad_norm': 24.91100311279297, 'learning_rate': 1.6019417475728158e-05, 'epoch': 2.84}
{'loss': 0.0001, 'grad_norm': 0.021272089332342148, 'learning_rate': 1.116504854368932e-05, 'epoch': 2.89}
{'loss': 0.1966, 'grad_norm': 0.0030857149977236986, 'learning_rate': 6.310679611650486e-06, 'epoch': 2.94}
{'loss': 0.0002, 'grad_norm': 0.004338894505053

In [26]:
from textattack.datasets import Dataset

# Create a custom dataset for TextAttack
class CustomDataset(Dataset):
    def __init__(self, reviews, labels):
        self.reviews = reviews
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.reviews[idx], self.labels[idx]

# Prepare the dataset for TextAttack
test_reviews = test_data['review'].tolist()
test_labels = test_data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0).tolist()

test_dataset_attack = CustomDataset(test_reviews, test_labels)

In [28]:
from sklearn.metrics import accuracy_score, f1_score

# Step 9.1: Evaluate on Original Test Set
test_tokens = tokenizer(test_reviews, padding='max_length', truncation=True, return_tensors="pt")
test_labels = torch.tensor(test_labels)

test_dataset = IMDBDataset(test_tokens, test_labels)
test_loader = DataLoader(test_dataset, batch_size=4)

# Define evaluation function
def evaluate(model, data_loader):
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
        for batch in data_loader:
            inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
            outputs = model(**inputs)
            preds.extend(torch.argmax(outputs.logits, axis=1).cpu().numpy())
            labels.extend(batch['labels'].cpu().numpy())
    return accuracy_score(labels, preds), f1_score(labels, preds)

# Evaluate the retrained model on original test set
accuracy, f1 = evaluate(model, test_loader)
print(f'Accuracy on original test set: {accuracy}')
print(f'F1 score on original test set: {f1}')

# Step 9.2: Evaluate on Adversarial Test Set
adversarial_test_examples = attacker.attack_dataset()  # Adjust this line

adversarial_test_reviews = [ex.perturbed_text() for ex in adversarial_test_examples]
adversarial_test_labels = [ex.original_result.output for ex in adversarial_test_examples]

# Tokenize adversarial test set
adversarial_test_tokens = tokenizer(adversarial_test_reviews, padding='max_length', truncation=True, return_tensors="pt")
adversarial_test_labels = torch.tensor(adversarial_test_labels)

adversarial_test_dataset = IMDBDataset(adversarial_test_tokens, adversarial_test_labels)
adversarial_test_loader = DataLoader(adversarial_test_dataset, batch_size=4)

# Evaluate the retrained model on adversarial test set
adv_accuracy, adv_f1 = evaluate(model, adversarial_test_loader)
print(f'Accuracy on adversarial test set: {adv_accuracy}')
print(f'F1 score on adversarial test set: {adv_f1}')

/var/folders/p1/5swwv58x6zx6zvpgsynmmnjm0000gn/T/ipykernel_77068/2310617606.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_labels = torch.tensor(test_labels)
/var/folders/p1/5swwv58x6zx6zvpgsynmmnjm0000gn/T/ipykernel_77068/2609057923.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/var/folders/p1/5swwv58x6zx6zvpgsynmmnjm0000gn/T/ipykernel_77068/2609057923.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[

Accuracy on original test set: 0.965
F1 score on original test set: 0.9637305699481865
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  15
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  20%|██        | 1/5 [04:27<17:49, 267.33s/it]

--------------------------------------------- Result 6 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (69%)]]

[[I]] rented I AM CURIOUS-YELLOW from my [[video]] store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this [[country]], therefore being a fan of [[films]] considered "controversial" I [[really]] had to see this for myself.<br /><br />The [[plot]] is centered around a young [[Swedish]] drama student named Lena who wants to learn everything she can about life. In [[particular]] she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and marri

[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:  40%|████      | 2/5 [15:39<23:29, 469.84s/it]

...

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (87%)]]

[[This]] [[film]] was probably inspired by Godard's Masculin, féminin and I urge you to see that film instead.<br /><br />The [[film]] has [[two]] strong elements and those are, (1) the realistic acting (2) the impressive, undeservedly good, [[photo]]. Apart from that, what strikes me most is the endless stream of silliness. Lena [[Nyman]] has to be most [[annoying]] [[actress]] in the [[world]]. [[She]] acts so stupid and with all the [[nudity]] in this film,...it's [[unattractive]]. [[Comparing]] to Godard's film, intellectuality has been replaced with stupidity. Without going too far on this subject, I would say that follows from the difference in ideals between the French and the Swedish society.<br /><br />A movie of its time, and place. 2/10.

[[Say]] [[teatro]] was probably inspired by Godard's Masculin, féminin and I urge you to see th

[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5: 100%|██████████| 5/5 [49:24<00:00, 592.88s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (51%)]]

Oh, brother...after hearing about this [[ridiculous]] [[film]] for umpteen years all I can think of is that [[old]] [[Peggy]] Lee [[song]]..<br /><br />"Is that all there is??" ...[[I]] was just an early teen when this smoked [[fish]] hit the U.S. [[I]] was too young to get in the theater ([[although]] [[I]] did manage to sneak into "Goodbye [[Columbus]]"). [[Then]] a screening at a [[local]] [[film]] [[museum]] [[beckoned]] - Finally I could see this [[film]], [[except]] now [[I]] was as old as my parents were when they schlepped to see it!!<br /><br />The ONLY reason this film was not condemned to the anonymous sands of [[time]] was because of the [[obscenity]] case sparked by its U.S. release. MILLIONS of [[people]] [[flocked]] to this stinker, thinking they were going to see a [[sex]] film...Instead, they got lots of closeups of [[gnarly]

/var/folders/p1/5swwv58x6zx6zvpgsynmmnjm0000gn/T/ipykernel_77068/2609057923.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/var/folders/p1/5swwv58x6zx6zvpgsynmmnjm0000gn/T/ipykernel_77068/2609057923.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Accuracy on adversarial test set: 0.5
F1 score on adversarial test set: 0.0
